# n개의 게시판 만들기
- category를 추가한 주소 이름 동적 생성!
- '<category>/<int:pk>/<mode>'   
- <font color= 'red'> render하거나 값을 넘겨줄 때 category값 꼭 추가</fonr>

In [ ]:
# base.html
'''
<h2>게시판 만들기</h2><br>

사용자: {{username}}<br>

<a href="/myboard/data/0/list"> 자료실</a>
<a href="/myboard/common/0/list"> 일반 게시판</a>
<a href="/myboard/etc/0/list"> 기타</a>


{% block content %}
{% endblock %}



<br>
<br>
<br>

copy right.....<br>
서울 특별시


'''

In [ ]:
# uels.py: url short cut
'''
from django.urls import path, redirect
from . import views 
from django.shortcuts import redirect


urlpatterns = [
    path('', views.index),
    path('<category>/<int:pk>/<mode>', views.BoardView.as_view(), name = 'myboard'),
    path('', lambda request: redirect('myboard', 'common', 0, 'list'),
    )
]


'''

In [ ]:
# views.py
'''
from django.shortcuts import render, get_object_or_404,redirect
from django.http import HttpResponse
from django.views.generic import View
from django.contrib.auth import authenticate
from django.contrib.auth.models import User
from django.forms import CharField, Textarea, ValidationError
from django import forms
#from blog.forms import PostForm  # 프로젝트 명이 바뀌면 blog를 바꿔야하므로 좋지 않음
from . import models
from . import forms
from . import apps

# url: bolg -> index함수 실행
def index(request):
    return HttpResponse('ok')

### <int:pk>/ <mode>를 url을 사용하기 위해
class BoardView(View) :
    def get(self, request, category, pk, mode):
        print(pk, mode)
        if mode =='add':
            # pk값은 상관없음.
            form = forms.BoardForm()
            return render(request, 'myboard/edit.html', {'form': form})
        elif mode == 'list':
            # pk값 상관없음.
            username = request.session.get('username', '')
            user = User.objects.get(username =username)
            data= models.Board.objects.all().filter(author = user, category = category)
            context = {'data': data,'username': username, 'category': category}
            return render (request, apps.APP + '/list.html', context)

        elif mode =='detail':
            p = get_object_or_404(models.Board, pk = pk)
            # 조회수 증가
            p.cnt +=1
            p.save()
            return render(request,  apps.APP + '/detail.html', {'d': p, 'category': category})

        elif mode == 'edit':
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(instance=post)

        elif mode =='delete':
            post = get_object_or_404(models.Board, pk=pk)
            post.delete()
            return redirect(category +'/0/list')
        else:
            return HttpResponse('mode를 잘 입력하세용')
        return render(request, apps.APP +'/edit.html', {"form":form}) # 정확하게 입력



    def post(self, request, category, pk, mode):

        username = request.session["username"]
        user = User.objects.get(username=username)

        if pk == 0:
            form = forms.BoardForm(request.POST)
        else:
            # 바뀌기 전 데이터
            post = get_object_or_404(models.Board, pk=pk)
            # 바뀌기 전 데이터로부터 form을 생성
            form = forms.BoardForm(request.POST, instance=post)

        if form.is_valid():
            # 바뀐 데이터를 저장은 하지 말고 post에 갖고 오기
            post = form.save(commit=False)
            # add할 때 저자정보 넣고
            if pk == 0:
                post.author = user
                post.category = category
            post.save()
            return redirect("myboard", category, 0, 'list')  
        # return render(request, "myboard/edit.html", {"form": form})
        return render(request, apps.APP + '/edit.html', {"form": form})
            
          


'''

In [ ]:
# list.html
'''
<!-- import, includ 개념 -->
{% extends 'myboard/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}
    <br>
   
    <a href="{% url 'myboard'  category 0 'add' %}">Add </a> <br>
    여기부터 sub-template<br>

    {% for d in data %}
    
            <a href="{% url 'myboard' category d.pk 'detail' %}"> {{d.title}} 조회수 {{d.cnt}}</a>
       <br>
    {% endfor %}


    

{% endblock %}



'''

In [ ]:
### detail.html
'''
<!-- import, includ 개념 -->
{% extends 'myboard/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

게시물을 보여줘<br>

<!-- {{ form.as_p }} -->
{{d.title}}
{{d.tedt|linebreaks}}   

    <a href="{%  url 'myboard' category d.pk 'edit'  %}">수정</a>

{% endblock %}

'''

In [ ]:
# apps.py
'''
from django.apps import AppConfig


APP = 'myboard'

class MyboardConfig(AppConfig):
    name = 'myboard'

'''

In [ ]:
# admin.py
'''
from django.contrib import admin
from . import models
# Register your models here.

admin.site.register(models.Board)
'''